# Time Series Forecasting

<span>This notebook demonstrates my solution to a time series problem posted on Kaggle. In the dataset, we have near 1 million training records from 1000 stores. We will need to use this information to predict the total Sales for each Day in the test set, which is about 40,000 records. During the modeling process, I created a few benchmark models to compare my LSTM Neural Network's performance against near the very end.</span>

**Dataset**

Rossman Dataset: https://www.kaggle.com/c/rossmann-store-sales/overview

### Import Preliminaries

In [2]:
# Import generic data science packages
%matplotlib inline
%config InlineBackend.figure_format='retina'

# Import modules
import datetime
import keras
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import missingno as msno
import numpy as np
import pandas as pd
import sqlalchemy
import sklearn
import seaborn as sns
import warnings

from keras import models, layers
from keras.wrappers.scikit_learn import KerasClassifier
from scipy.sparse import csr_matrix
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Set pandas options
pd.set_option('max_columns',1000)
pd.set_option('max_rows',30)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Set plotting options
mpl.rcParams['figure.figsize'] = (8.0, 7.0)

ModuleNotFoundError: No module named 'keras'

### Import Data

The training and testing set have been given to us into separate CSV files, with an external file for the additional store data. I found empty string values in the data after importing the store table, so I wrote a quick "for" loop in the cell below, changing these empty string to null values. We will address how to handle the null values later on.

In [ ]:
# Set your local database parameters 
db_username = 'root'
db_password = 'mypassword'
host = '127.0.0.1'
port = '3306'
db_name = 'rossman'

# Create a MySQLEngine
engine = sqlalchemy.create_engine('mysql+mysqldb://'+db_username+':'+db_password+'@\
'+host+':'+port+'/'+db_name)

# Connect to database
engine.connect();

# Import data from SQL
train_df = pd.read_sql(""" SELECT * FROM train""", engine,
                      parse_dates=['Date'],
                      ).dropna(how='any', axis=0)
test_df = pd.read_sql(""" SELECT * FROM test""", engine,
                     index_col='Id', parse_dates=['Date']
                     ).dropna(how='any', axis=0)
stores = pd.read_sql(""" SELECT * FROM store""", engine,
                     ).dropna(how='any', axis=0)

# Rename salse column to target
train_df.rename(columns={'Sales':'target'}, inplace=True)

# Replace empty string in store dataframe with Null Values
for col in stores:
    stores[col] = stores[col].replace('',np.nan)

# Down Sample Dataset
#train_df = train_df.sample(10_000)
#test_df = test_df.sample(10_000)
    
# View Store Dataframe
stores.head(5)

In [ ]:
# View head of the dataframe
test_df.head(10)

In [ ]:
# View head of the dataframe
train_df.head(10)

### Data Completness

"MissingNo" is a library that makes it easy to plot the completeness of your dataset. The training and testing data frames do not have any null values, but the store dataframe is missing a few data points in a couple of features. Remember these were the empty string values that we changed to null values.

In [ ]:
# Plot the null values from train, tets, and store dataframes
msno.matrix(train_df,  figsize=(8,6), fontsize=10);
plt.xlabel('Feature')
plt.ylabel('Observations');

msno.matrix(test_df,  figsize=(8,6), fontsize=10);
plt.xlabel('Feature')
plt.ylabel('Observations');

msno.matrix(stores,  figsize=(8,6), fontsize=10);
plt.xlabel('Feature')
plt.ylabel('Observations');

### Data Overview

In [ ]:
# Plot the distribution of target classes
train_df.target.plot.hist(figsize=(9,5),color='#BAE4B3', edgecolor='black')
plt.axvline(train_df.target.mean(), color='r', linestyle='dashed', 
            linewidth=2);
plt.title('Sales Histogram')
plt.ylabel('Salse')
plt.xlabel('Number of Records');

In [ ]:
# Plot a barplot of the target clasees in our training set
# (1 = posinous, 0 = eatable by humans)
train_df.DayOfWeek.value_counts().sort_index().plot.barh(
                figsize = (9,5),
                grid=False, 
                color=['#edf8e9','#c7e9c0','#a1d99b','#74c476',
                  '#41ab5d','#238b45','#005a32'], 
                width=0.5, edgecolor='black',
                hatch='|')
plt.title('Target Outcomes')
plt.ylabel('Target Class')
plt.xlabel('Number of Records');

In [ ]:
# Plot counts for days in the dataset a store is open
plt.figure()
open_counts = train_df.Open.value_counts()
open_counts.plot.barh(figsize=(9,3),grid=False, 
                      color=['#CFCCC7','#3F3F3F'],
                      width=0.25,edgecolor='w')
plt.title('Open Counts')
plt.ylabel('Open Classes')
plt.xlabel('Number of Records');

# Plot number of time that we are open during state holidary (probably never)
plt.figure()
sthol_counts = train_df.StateHoliday.value_counts()
sthol_counts = pd.Series([sthol_counts.values[0], 0 ]) # plotting fix
sthol_counts.plot.barh(figsize=(9,3),grid=False, 
                       color=['#CFCCC7','#3F3F3F'], 
                       width=0.25,edgecolor='w')
plt.title('Stat Holiday Counts')
plt.ylabel('Stat Holiday Classes')
plt.xlabel('Number of Records');

# Plot the number of days in the datset that are affected by school holidays
plt.figure()
schol_counts = train_df.SchoolHoliday.value_counts()
schol_counts.plot.barh(figsize=(9,3),grid=False, 
                       color=['#CFCCC7','#3F3F3F'], 
                       width=0.25,edgecolor='w')
plt.title('School Holiday Counts')
plt.ylabel('Target Class')
plt.xlabel('Number of Records');

In [ ]:
# plot sales by day of the week
sales_by_week = train_df.groupby('DayOfWeek')['target'].mean()
sales_by_week.plot.barh(
                figsize = (9,5),
                grid=False, 
                color=['#edf8e9','#c7e9c0','#a1d99b','#74c476',
                  '#41ab5d','#238b45','#005a32'], 
                width=0.5, edgecolor='black',
                hatch='o')
plt.title('Average Salse Per Day of Week')
plt.ylabel('Day of Week')
plt.xlabel('Sales');


In [ ]:
# Plot a line plot showing growth by year
annual_growth = train_df.set_index('Date')
annual_growth = annual_growth.resample('1M').sum()
annual_growth.plot.area(y='target', figsize=(15,5), 
                        color='#A1D99B', alpha=0.5)
plt.xticks(rotation='vertical')
plt.title('Sales over Time')
plt.xlabel('Datetime')
plt.ylabel('Sales');

### Data Incosistencies

In [ ]:
# Print the featuer names from each dataframe
print('Training Dataset:', train_df.columns, '\n')
print('Testing Dataset:', test_df.columns)

We can see that training dataset has a column that is not present in our Testing Set. Therefore let's map the customers count to the storse dataset than merge the total customer counts back into the main testing and training dataframes in the next cells.

In [ ]:
# Store customer information seperately
customers_df = train_df[['Store','Customers']]
customers_df = customers_df.groupby('Store')['Customers'].sum().to_frame()
customers_df.rename(columns={'Customers':'Lifetime Customers'}, inplace=True)
train_df = train_df.drop('Customers', axis=1)
customers_df.head(10)

### Data Cleaning

In [ ]:
# Copy dataframe for cleaning and feature engineering
train_fdf = train_df.copy()
test_fdf = test_df.copy()
store_fdf = stores.copy()

train_fdf = train_fdf.merge(stores, how='left', on='Store')
test_fdf = test_fdf.merge(stores, how='left', on='Store')

# Deal will null values from merge
train_fdf.CompetitionOpenSinceMonth.fillna(-1, inplace=True)
train_fdf.CompetitionOpenSinceYear.fillna(datetime.datetime.now().year + 1, inplace=True)
train_fdf.Promo2.fillna(-1, inplace=True)
train_fdf.Promo2SinceWeek.fillna(-1, inplace=True)
train_fdf.Promo2SinceYear.fillna(datetime.datetime.now().year + 1, inplace=True)
train_fdf.PromoInterval.fillna('Unknown', inplace=True)

# Deal will null values after merge
train_fdf.Assortment.fillna('Unknown', inplace=True)
train_fdf.StoreType.fillna('Uknown', inplace=True)
train_fdf.CompetitionDistance.fillna(train_fdf.CompetitionDistance.mean(), inplace =True)

# View existing null value of the train dataframe
train_fdf.isnull().sum()

In [ ]:
# Deal will null values from merge
test_fdf.CompetitionOpenSinceMonth.fillna(-1, inplace=True)
test_fdf.CompetitionOpenSinceYear.fillna(datetime.datetime.now().year + 1, inplace=True)
test_fdf.Promo2.fillna(-1, inplace=True)
test_fdf.Promo2SinceWeek.fillna(-1, inplace=True)
test_fdf.Promo2SinceYear.fillna(datetime.datetime.now().year + 1, inplace=True)
test_fdf.PromoInterval.fillna('Unknown', inplace=True)

# Deal will null values after merge
test_fdf.Assortment.fillna('Unknown', inplace=True)
test_fdf.StoreType.fillna('Uknown', inplace=True)
test_fdf.CompetitionDistance.fillna(train_fdf.CompetitionDistance.mean(), inplace =True)

# View existing null value of the test dataframe
test_fdf.isnull().sum()

In [3]:
# Plot feature correlation matrix
plt.figure(figsize=(9,9))
sns.heatmap(train_fdf.corr(), cmap=plt.cm.Greens, linewidth=1)
plt.title('Feature Correlation Heatmap (Before Feature Engineering)')
plt.xlabel('Features')
plt.ylabel('Features');

NameError: name 'plt' is not defined

The strange correlation between StateHoliday and every other feature in the training set is because the StateHoliday feature is an entire feature with just the value "0". Therefore we will remove this feature at the tail end of our feature engineering.

### Feature Engineering

In [16]:
# Append Customer Information
train_fdf = train_fdf.merge(customers_df, how='left', on='Store')
test_fdf = test_fdf.merge(customers_df, how='left', on='Store')


# One Hot Encoding Train Non-Ordinal Feature
train_fdf = pd.get_dummies(train_fdf, drop_first=True,
               columns=['Assortment','Store','StoreType','Promo2',
                        'Promo2SinceYear','PromoInterval'])

# One Hot Encoding Test Non-Ordinal Feature
test_fdf = pd.get_dummies(test_fdf, drop_first=True,
               columns=['Assortment','Store','StoreType','Promo2',
                        'Promo2SinceYear','PromoInterval'])

# Expand Train DateTime Feature
train_fdf['Year'] = train_fdf['Date'].dt.year
train_fdf['Day'] = train_fdf['Date'].dt.day
train_fdf['Montah']= train_fdf['Date'].dt.month
train_fdf.drop('Date', axis=1 ,inplace=True)

# Expand Test DateTime Feature
test_fdf['Year'] = test_fdf['Date'].dt.year
test_fdf['Day'] = test_fdf['Date'].dt.day
test_fdf['Month']= test_fdf['Date'].dt.month
test_fdf.drop('Date', axis=1 ,inplace=True)

# Change the Opent Store Year into a Year Ago
train_fdf.CompetitionOpenSinceYear = (train_fdf.CompetitionOpenSinceYear.astype(int) 
                                      - datetime.datetime.now().year)
test_fdf.CompetitionOpenSinceYear = (test_fdf.CompetitionOpenSinceYear.astype(int) 
                                      - datetime.datetime.now().year)
# Drop StateHoliday Feature
train_df.drop('StateHoliday', axis=1, inplace=True )
test_df.drop('StateHoliday', axis=1, inplace=True)

### Encoding + Split Data

In [17]:
# Clean us some remaining object dtypes into ints
for df in [train_fdf, test_fdf]:
    for col in df.select_dtypes(include='object'):
        df[col] = df[col].astype(int)

# Target Values
y = train_fdf.target
X = train_fdf.drop('target', axis=1)
print(f'Training Data Shape:',X.shape)
print(f'Training Target Shape:',y.shape)

# Create KFold cross validation rule without shuffling since we are 
# dealing with a time series
kfold = KFold(n_splits=10, shuffle=False)

Training Data Shape: (986159, 1146)
Training Target Shape: (986159,)


### Benchmark Model Pipelines

In [273]:
# Linear Regression Pipeline 
lr_pipeline = make_pipeline(PCA(n_components=10),StandardScaler(), LinearRegression())
lr_scores = cross_val_score(lr_pipeline, X, y, cv=kfold, scoring='neg_mean_absolute_error')
print(f'Linear Regression Cross Validation Score - Mean Absolute Erorr: {lr_scores.mean():.5f}')

Linear Regression Cross Validation Score - Mean Absolute Erorr: -2108.44595


In [274]:
# Linear Regression Pipeline 
rfr_pipeline = make_pipeline(PCA(n_components=10),StandardScaler(), RandomForestRegressor())
rfr_scores = cross_val_score(rfr_pipeline, X, y, cv=kfold, scoring='neg_mean_absolute_error')
print(f'Linear Regression Cross Validation Score - Mean Absolute Erorr: {rfr_scores.mean():.5f}')

Linear Regression Cross Validation Score - Mean Absolute Erorr: -779.27056


In [23]:
# Dense Neural Network
def create_ds_model():
    model = models.Sequential()
    model.add(layers.Dense(128, activation='relu', input_shape=(10,)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mean_absolute_error', metrics=['mae'])
    return model

ds_model = KerasClassifier(build_fn=create_ds_model, epochs=50, batch_size=10, verbose=0)

# Linear Regression Pipeline 
ds_pipeline = make_pipeline(PCA(n_components=10),StandardScaler(), ds_model)
ds_scores = cross_val_score(ds_pipeline, X, y, cv=kfold, scoring='neg_mean_absolute_error')
print(f'Densly Connected Cross Validation Score -  Mean Absolute Error : {ds_scores.mean():.5f}')

Densly Connected Cross Validation Score -  Mean Absolute Error : -5905.11181


### LSTM Model Pipeline

In [ ]:
def create_lstm_model():
    model = models.Sequential()
    model.add(layers.LSTM(units=128))
    model.add(layers.Dense(units=1, activation='sigmoid'))
    model.compile(loss='mean_absolute_error', # Cross-entropy
                optimizer='Adam', # Adam optimization
                metrics=['mse']) # Accuracy performance metric
    return model

lstm_model = KerasClassifier(build_fn=create_lstm_model, epochs=50, batch_size=10, verbose=1)
lstm_pipeline = make_pipeline(PCA(n_components=10),StandardScaler(), lstm_model)
lstm_scores = cross_val_score(lstm_pipeline, X, y, cv=kfold, scoring='neg_mean_absolute_error')
print(f'Densly Connected Cross Validation Score -  Mean Absolute Error : {lstm_scores.mean():.5f}')

In [27]:
lstm_model.summary()

AttributeError: 'KerasClassifier' object has no attribute 'summary'

### Final Model

I will be using the last LSTM model as my final model for this problem. Generally at this stage, we would begin tuning the hyperparameters via Grid Search, but for the sake of not destroying my local computer, we will leave it here for now. 

In [ ]:
# PCA and scale the entire dataset
decomposer = PCA(n_components=10)
X_decomposed = decomposer.fit_transform(X)
scaler = StandardScaler()
X_dec_scal = scaler.fit_transform(X_decomposed)

# Build LSTM Model again but on the entire dataset
model = models.Sequential()
model.add(layers.Embedding(input_dim=train_x.shape[1], output_dim=128))
model.add(layers.LSTM(units=128))
model.add(layers.Dense(units=1, activation='sigmoid'))
model.compile(loss='neg_mean_absolute_error', # Cross-entropy
            optimizer='Adam', # Adam optimization
            metrics=['mse']) # Accuracy performance metric
model.fit(X_des_scal, y)

Author: Kavi Sekhon